In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import json

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, reg_ex,clean_data, curate_kim     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '137'
formatted_attribute = 'Length'
customer_name='%kimballmidwest%'
buckets = """Office Chairs"""

strategy_version_input=751
attribute_id_input=2425

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['Length']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df=dfs
print(len(df))
# reg_ex(df)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']


14241


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [6]:
reg_ex(df)
print(len(df))

14241


In [7]:
dfna=df[df['mm_long'].astype(str)=='[]']
print(len(dfna))
dfna[curation_col]='n/a'
matchna=dfna[['external_id',curation_col]]
df=df[df['mm_long'].astype(str)=='[]']

13681


In [8]:
print(len(df[(df['Length'].astype(str)!='[]')&(df['Length'].astype(str)!='nan')]))
length_curation=df[(df['Length'].astype(str)!='[]')&(df['Length'].astype(str)!='nan')]
length_curation['lengths']=length_curation['Length'].apply(lambda x: re.sub(r'.*"\s?\-','',str(x)))
length_curation['lengths']=length_curation['lengths'].astype(str).apply(lambda x: re.sub(r',',"','",str(x)))
clean_data(length_curation,'lengths','length_final')

length_curation['length_final']=length_curation['length_final'].apply(lambda x: f"['{x}']").apply(lambda x: re.sub(r',','","',str(x)))
length_curation['length_final']=length_curation['length_final'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x)))
rounds(length_curation,'length_final')
rounding(length_curation,'length_final','a-eghj-mo-su-z')
length_curation[curation_col]=length_curation['length_final']
length_curation=length_curation[length_curation['rounding'].astype(str)=='[]']

pat_range=rf'''(?i)("\s?\-)|()'''         
length_curation['range'] = length_curation['Length'].apply(lambda x: re_extract(pat_range, x))

matchlencur=length_curation[['external_id',curation_col]]
lengths=curate_kim(df,'Length','length',formatted_attribute)
matchlens=lengths[['external_id',curation_col]]

1185
30


# Tripples

In [9]:
tripple=df[['external_id','blank_matches_long','tripple_name','tripple_long','double_name','double_long']]

pat_mm=rf'''(?i)(mm)|()'''         
tripple['mm'] = tripple['tripple_name'].apply(lambda x: re_extract(pat_mm, str(x)))

tripple=tripple[(tripple['tripple_name'].astype(str)!='[]')&(tripple['mm'].astype(str)=='[]')]
tripple['tripple_name']=tripple['tripple_name'].apply(lambda x: re.sub(r'(?i)x.*','',str(x)))
tripple['tripple_name']=tripple['tripple_name'].astype(str).apply(lambda x: re.sub(r',',"','",str(x)))
clean_data(tripple,'tripple_name','tripple')
# # curate_fractions(length_curation,'length_final',formatted_attribute,'a-eghj-mo-su-z')
tripple['trip']=tripple['tripple'].apply(lambda x: f"['{x}']").apply(lambda x: re.sub(r',','","',str(x)))
tripple['trip']=tripple['trip'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x)))
rounds(tripple,'trip')
rounding(tripple,'trip','a-eghj-mo-su-z')
tripple[curation_col]=tripple['trip']
tripple=tripple[tripple['rounding'].astype(str)=='[]']
tripple[curation_col]=tripple[curation_col].apply(lambda x: re.sub(r'\["\["','["',str(x))).apply(lambda x: re.sub(r'\/(?!\d)','',str(x)))
tripple=tripple[(tripple[curation_col].astype(str)!='[""]')&(tripple[curation_col].astype(str)!='["i"]')]
print(len(tripple))
tripple[0:500]

4


external_id blank_matches_long tripple_name tripple_long double_name  \
3429       93404                 []        ['1"            []   [1" x 1"]   
3709       93400                 []        ['1"            []   [1" x 1"]   
3715       93402                 []        ['1"            []   [1" x 1"]   
4142       93866                 []        ['1"            []   [1" x 1"]   

     double_long  mm tripple        trip rounding  Q:length  
3429          []  []  ['1 in  ["["1 in"]       []  ["1 in"]  
3709          []  []  ['1 in  ["["1 in"]       []  ["1 in"]  
3715          []  []  ['1 in  ["["1 in"]       []  ["1 in"]  
4142          []  []  ['1 in  ["["1 in"]       []  ["1 in"]

In [10]:
matchtripple=tripple[['external_id',curation_col]]

In [11]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchlencur))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchlens))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchlens))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))

matchlencur
matchlens
matchlens
matchna


In [12]:
print(len(df))
print(len(matchlencur)+len(matchlens)+len(matchna))

13681
14724


In [13]:
stop   

NameError: name 'stop' is not defined

# send to the folder for upload

In [15]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchlens)

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchlencur)    

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchlencur) 

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 